# `bw_temporalis` Introduction

We want to know when things occur, so that we can see how long it takes investments in clean energy to pay, or so that we can discount emissions from events which have already taken place.

## [`TemporalDistribution` (source code)](https://github.com/brightway-lca/bw_temporalis/blob/6517a0677d0e854d41eed1155df388e56837207d/bw_temporalis/temporal_distribution.py#L17)

To do this we need several building blocks. The first is a data structure to locate things in time. We call this a `TemporalDistribution`, and it has two parts: the dates things occur, and the amount that occurs at each step. A `TemporalDistribution` is quantized - it is a set of points, not a smooth function.

In [ ]:
from bw_temporalis import easy_timedelta_distribution, easy_datetime_distribution, TemporalisLCA, Timeline, TemporalDistribution
from bw_temporalis.lcia import characterize_methane, characterize_co2
import bw2data as bd
import bw2calc as bc
import bw_graph_tools as graph
import numpy as np
import pandas as pd

In [ ]:
td = easy_timedelta_distribution(
    start=0,
    end=10, # Range includes both start and end
    resolution="h",  # M for months, Y for years, h for hours, etc.
    steps=5,
    kind="triangular",
    param=3,
)

We can plot this with `pyplot`, but it shows things in seconds:

In [ ]:
td.graph()

A `TemporalDistribution` can also be absolute instead of relative:

In [ ]:
one_day = easy_datetime_distribution("2023-05-23", "2023-05-24", steps=10)
one_day.graph()

The cool thing about temporal distributions is that we can combine them. Specifically, we can:

* Multiply or divide by a number (changes only `amount`)
* Multiply (convolve) an absolute and a relative temporal distribution
* Multiply (convolve) two relative temporal distributions
* Add an absolute and a relative temporal distribution (what is the result?)
* Add two relative temporal distributions (concatenates the arrays)

Every mathematical operation produces a new temporal distribution.

In [ ]:
(one_day * td).graph()

The above graph looks a bit weird, but it is correct. Can you figure out why there aren't five lines of points?

Let's do a simpler example to see how discrete convolution works. We can take two temporal distributions, each with only three values:

```python
a = TemporalDistribution(
    date=np.array([-1, 0, 1], dtype='timedelta64[M]'),  # `M` is months
    amount=np.array([0.2, 0.6, 0.2])
)
b = TemporalDistribution(
    date=np.array([-2, -1, 0], dtype='timedelta64[M]'),
    amount=np.array([0.5, 0.3, 0.2])
)
```

Discrete convolution is just *adding* each possible combination of relative or absolute dates, *multiplying* each respective possible combination of amounts, and then combining them in the correct order while addint together duplicate values.

Here are our dates:

| | -1 (a0) | 0 (a1) | 1 (a2) |
| --- | --- | --- | --- |
| -2 (b0) | -3 | -2 | -1 |
| -1 (b1) | -2 | -1 | 0 |
| 0 (b2) | -1 | 0 | 1 |

And our amounts:

| | 0.2 (a0) | 0.6 (a1) | 0.2 (a2) |
| --- | --- | --- | --- |
| 0.5 (b0) | 0.1 | 0.3 | 0.1 |
| 0.3 (b1) | 0.06 | 0.18 | 0.06 |
| 0.2 (b2) | 0.04 | 0.12 | 0.04 |

We need to combine the amounts that occur at the same time:

| When | -3 | -2 | -1 | 0 | 1 |
| --- | --- | --- | --- | --- | --- |
| What | 0.1 | 0.06 + 0.3 | 0.04 + 0.18 + 0.1 | 0.12 + 0.06 | 0.04 |

And we get our new distribution:

| When | -3 | -2 | -1 | 0 | 1 |
| --- | --- | --- | --- | --- | --- |
| What | 0.1 | 0.36 | 0.32 | 0.18 | 0.04 |


In [ ]:
a = TemporalDistribution(
    date=np.array([-1, 0, 1], dtype='timedelta64[M]'),  # `M` is months
    amount=np.array([0.2, 0.6, 0.2])
)
b = TemporalDistribution(
    date=np.array([-2, -1, 0], dtype='timedelta64[M]'),
    amount=np.array([0.5, 0.3, 0.2])
)
c = a * b
c.date.astype('timedelta64[M]'), c.amount

Note that the total amount stays the same: it was one for both `a` and `b`, and is the same for `c`.

In [ ]:
c.amount.sum()

You won't really understand all this unless you play with it! Please do the following:

* Add a random number to both `td` and `one_day` and graph the results. Use `a` and `b` to insert new cells in the notebook.
* Construct another *relative* (i.e. `timedelta`) temporal distribution, (use `c` and `v` keys to copy and paste the existing cell), and multiply the two relative temporal distributions. Graph the result. Do you understand the result?
* *Add* the two relative distributions and see what happens. Can you see why we don't use addition in Temporalis?
* Construct a new *absolute* (i.e. `datetime`) distribution, and multiply it by ony of your relative distributions. Work out the expected result by hand, and see if you get the correct answer.

## [`NewNodeEachVisitGraphTraversal` (source code)](https://github.com/brightway-lca/bw_graph_tools/blob/main/bw_graph_tools/graph_traversal.py#L159)

Our supply chain graphs have cycles, and we need to decide how to handle these cycles when doing temporally-aware graph traversal. Here is a simple example:

<img src="images/circle.png">

There are two ways to handle this in graph traversal:

* We only visit each node once - when we encounter it again, we ignore it, as we have already visited it.
* Every visit to a node is treated separately

See also [Are there infinitely many trucks in the technosphere, or exactly one?](https://link.springer.com/article/10.1007/s11367-018-1519-8).

We have to follow approach two in temporally-aware graph traversal, because each *time* we visit a node it is a different *time* (not sure this is a dad joke or just awkward working :p). That's why the name of our graph traversal class is so long: `NewNodeEachVisitGraphTraversal`. Each time we see a node in the graph, we create a [new instance of a `Node` dataclass](https://github.com/brightway-lca/bw_graph_tools/blob/b2309b2fdaac46f586117d9e45df5ffd0769e31e/bw_graph_tools/graph_traversal.py#L52), and we store a bunch of data, including:

* unique_id : A unique integer id for this visit to this activity node
* activity_datapackage_id : The id that identifies this activity in the datapackage, and hence in the database
* supply_amount : The amount of the *activity* (not reference product!) needed to supply the demand from the requesting supply chain edge.
* cumulative_score : Total LCIA score attributed to `supply_amount` of this activity. Includes direct emissions unless explicitly removed.
* direct_emissions_score : Total LCIA score attributed only to the direct characterized biosphere flows of `supply_amount` of this activity.

So, if in our simple example we started with `A`, we would add `A(unique_id=0)`, then follow the link to `B(unique_id=1)`, then `A(unique_id=2)`, then `B(unique_id=3)`, etc., until we hit some cut-off criteria. Our graph traversal algorithm is *priority-first* - that means we follow the next available edge in the supply chain with the highest impact. Traditional graph traversal is normally breadth-first:

<img src="images/breadth.png">

Or depth-first:

<img src="images/depth.png">

Breadth-first is very inefficient, as in a real background database we would quickly reach millions of edges with no real contributions, while depth first doesn't make sense as our graph is highly cyclic, so there is no "bottom" layer. Instead, we let the LCIA scores tell us what next to calculate:

<img src="images/priority.png">

This is a bit complicated, especially if you are new to thinking in code; you can read the source code and [the tests](https://github.com/brightway-lca/bw_graph_tools/tree/b2309b2fdaac46f586117d9e45df5ffd0769e31e/tests/traversal) to see the results of a graph traveral algorithm.

## [`Timeline` (source code)](https://github.com/brightway-lca/bw_temporalis/blob/6517a0677d0e854d41eed1155df388e56837207d/bw_temporalis/timeline.py#L37)

After we do graph traversal, and convolve our temporal distributions whenever we meet them, we have a disorganized pile of data. We can assemble the distributions together into a timeline of when emissions occur, sorted by time, and noting both the emission id and the id of the activity which caused the emission.

Let's make an example system to see this in action. Thanks to Giuseppe Cardellini for this example data.

In [ ]:
bd.projects.set_current("Temporalis example project")

In the following code, I use both `TemporalDistribution` and `easy_timedelta_distribution`. I recommend using the `easy` functions when possible, as it will help prevent you from making silly mistakes. For example, when preparing this notebook I made multiple silly mistakes in not making sure my temporal distributions summed to (positive) one, and lost a bunch of time. If you use `TemporalDistribution`, you can run the utility function `bw_temporalis.check_database_exchanges` to make sure everything is the way its supposed to be.

In [ ]:
bd.Database('temporalis-example').write({
    ('temporalis-example', "CO2"): {
        "type": "emission",
        "name": "carbon dioxide",
        "temporalis code": "co2",
    },
    ('temporalis-example', "CH4"): {
        "type": "emission",
        "name": "methane",
        "temporalis code": "ch4",
    },
    ('temporalis-example', 'Functional Unit'): {
        'name': 'Functional Unit',
        'exchanges': [
            {
                'amount': 5,
                'input': ('temporalis-example', 'EOL'),
                'temporal_distribution': easy_timedelta_distribution(
                    start=0,
                    end=4, # Range includes both start and end
                    resolution="Y",  # M for months, Y for years, etc.
                    steps=5,
                ),
                'type': 'technosphere'
            },
        ],
    },
    ('temporalis-example', 'EOL'): {
        'exchanges': [
            {
                'amount': 0.8,
                'input': ('temporalis-example', 'Waste'),
                'type': 'technosphere'
            },
            {
                'amount': 0.2,
                'input': ('temporalis-example', 'Landfill'),
                'type': 'technosphere'
            },
            {
                'amount': 1,
                'input': ('temporalis-example', 'Use'),
                'type': 'technosphere'
            },
        ],
        'name': 'EOL',
        'type': 'process'
    },
    ('temporalis-example', 'Use'): {
        'exchanges': [
            {
                'amount': 1,
                'input': ('temporalis-example', 'Production'),
                'temporal_distribution': TemporalDistribution(
                    np.array([-4], dtype='timedelta64[M]'),
                    np.array([1.0])
                ),
                'type': 'technosphere'
            },
        ],
        'name': 'Use',
    },
    ('temporalis-example', 'Production'): {
        'exchanges': [
            {
                'amount': 1,
                'input': ('temporalis-example', 'Transport'),
                'temporal_distribution': TemporalDistribution(
                    np.array([200],dtype='timedelta64[D]'),
                    np.array([1.0])
                ),
                'type': 'technosphere'
            },
        ],
        'name': 'Production',
        'type': 'process'
    },
    ('temporalis-example', 'Transport'): {
        'exchanges': [
            {
                'amount': 1,
                'input': ('temporalis-example', 'Sawmill'),
                'type': 'technosphere'
            },
            {
                'amount': 0.1,
                'input': ('temporalis-example', 'CO2'),
                'type': 'biosphere'
            },
        ],
        'name': 'Production',
        'type': 'process'
    },
    ('temporalis-example', 'Sawmill'): {
        'exchanges': [
            {
                'amount': 1.2,
                'input': ('temporalis-example', 'Forest'),
                'temporal_distribution': TemporalDistribution(
                    np.array([-14], dtype='timedelta64[M]'),
                    np.array([1.0])
                ),
                'type': 'technosphere'
            },
            {
                'amount': 0.1,
                'input': ('temporalis-example', 'CO2'),
                'type': 'biosphere'
            },
        ],
        'name': 'Sawmill',
        'type': 'process'
    },
    ('temporalis-example', 'Forest'): {
        'exchanges': [
            {
                'amount': -.1 * 6,
                'input': ('temporalis-example', 'CO2'),
                'temporal_distribution': TemporalDistribution(
                    np.array([-4, -3, 0, 1, 2, 5], dtype='timedelta64[Y]'),
                    np.ones(6) * (1/6)
                ),
                'type': 'biosphere'
            },
            {
                'amount': 1.5,
                'input': ('temporalis-example', 'Thinning'),
                'temporal_distribution': TemporalDistribution(
                    np.array([-3, 0, 1], dtype='timedelta64[Y]'),
                    np.ones(3) * 1/3
                ),
                'type': 'technosphere'
            },
        ],
        'name': 'Forest',
    },
    ('temporalis-example', 'Thinning'): {
        'exchanges': [
            {
                'amount': 1,
                'input': ('temporalis-example', 'Thinning'),
                'type': 'production'
            },
            {
                'amount': 1,
                'input': ('temporalis-example', 'Avoided impact - thinnings'),
                'type': 'production'
            },
        ],
        'name': 'Thinning',
        'type': 'process'
    },
    ('temporalis-example', 'Landfill'): {
        'exchanges': [
            {
                'amount': 0.1,
                'input': ('temporalis-example', 'CH4'),
                'temporal_distribution': TemporalDistribution(
                    np.array([10, 20, 60, 100], dtype='timedelta64[M]'),
                    np.ones(4) * 1/4
                ),
                'type': 'biosphere'
            },
        ],
        'name': 'Landfill',
        'type': 'process'
    },
    ('temporalis-example', 'Waste'): {
        'exchanges': [
            {
                'amount': 1,
                'input': ('temporalis-example', 'Waste'),
                'type': 'production'
            },
            {
                'amount': 1,
                'input': ('temporalis-example', 'Avoided impact - waste'),
                'type': 'production'
            },
        ],
        'name': 'Waste',
        'type': 'process'
    },
    ('temporalis-example', 'Avoided impact - waste'): {
        'exchanges': [
            {
                'amount': -0.4,
                'input': ('temporalis-example', 'CO2'),
                'type': 'biosphere'
            },
            {
                'amount': 1,
                'input': ('temporalis-example', 'Avoided impact - waste'),
                'type': 'production'
            },
        ],
        'name': 'Avoided impact - waste',
        'type': 'process'
    },
    ('temporalis-example', 'Avoided impact - thinnings'): {
        'exchanges': [
            {
                'amount': -0.3,
                'input': ('temporalis-example', 'CO2'),
                'type': 'biosphere'
            },
            {
                'amount': 1,
                'input': ('temporalis-example', 'Avoided impact - thinnings'),
                'type': 'production'
            },
        ],
        'name': 'Avoided impact - thinnings',
        'type': 'process'
    }
})

The procedure to do temporal LCA is a bit more complicated than normal. We need to calculate the time-generic LCA using the `LCA` class, and then put this into the `TemporalisLCA`. 

We do it like this so that we can substitute cool new `LCA` classes (maybe regionalized LCA?) instead of only using the base `LCA` class.

In [ ]:
bd.Method(("GWP", "example")).write([
    (('temporalis-example', "CO2"), 1),
    (('temporalis-example', "CH4"), 25),
])

In [ ]:
lca = bc.LCA({('temporalis-example', 'EOL'): 1}, ("GWP", "example"))
lca.lci()
lca.lcia()

In [ ]:
lca = TemporalisLCA(lca)

In [ ]:
tl = lca.build_timeline()

When we first build the timeline, the data isn't organized or sorted. We need to put it into a dataframe if we want to make it nicer for human consumption:

In [ ]:
tl.build_dataframe()

We can lookup what the `id` values mean:

In [ ]:
bd.get_node(id=2), bd.get_node(id=11)

And graph the timeline:

In [ ]:
tl.df.plot(x="date", y="amount", kind="scatter")

## Characterization

Environmental impacts don't always occur at the time of emission, and aren't instantaneous. Temporalis includes some basic characterization for climate change, and we would be quite happy to have more temporally-aware LCIA methods included in the library!

In [ ]:
characterized_df_co2 = tl.characterize_dataframe(
    characterization_function=characterize_co2, 
    flow={bd.get_node(name="carbon dioxide").id},
)

In [ ]:
characterized_df_co2.plot(x="date", y="amount", kind="scatter")

In [ ]:
characterized_df_co2.plot(x="date", y="amount_sum", kind="scatter")

In [ ]:
characterized_df_ch4 = tl.characterize_dataframe(
    characterization_function=characterize_methane, 
    flow={bd.get_node(name="methane").id},
)

In [ ]:
characterized_df_ch4.plot(x="date", y="amount_sum", kind="scatter")

We can add these two characterized dataframes together to get net radiative forcing (in watts per square meter) over time:

In [ ]:
aggregate_df = pd.concat([characterized_df_ch4, characterized_df_co2])
aggregate_df.sort_values(by="date", ascending=True, inplace=True)
aggregate_df["amount_sum"] = aggregate_df["amount"].cumsum()

In [ ]:
axes = aggregate_df.plot(x="date", y="amount_sum")
axes.set_ylabel("Radiative forcing $\\frac{Watt}{m^{2}}$")
axes.set_xlabel("Time (years)")